# Deep Deterministic Policy Gradients (DDPG)
---
In this notebook, we train DDPG with OpenAI Gym's Pendulum-v0 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

from unityagents import UnityEnvironment

import pdb

### 2. Instantiate the Environment and Agent

In [2]:
path = 'C:\\Users\\mbgpcsk4\\Dropbox (The University of Manchester)\\D2019\\University\\Udacity\\P2_Reacher_Submission\\Udacity\\MultiAgent\\Reacher_Windows_x86_64\\Reacher.exe'

env = UnityEnvironment(file_name=path)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

agent = Agent(state_size=33, action_size=4, random_seed=2)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
agent.reset()
agent.print_params()

Actor:

None tensor(1.00000e-03 *
       [[ 1.0662,  0.6504, -1.7677,  ..., -2.8250, -1.7618, -2.4222],
        [-2.4825, -0.4878, -0.4335,  ..., -0.1893, -2.1118, -0.2032],
        [ 2.2218, -0.6816, -1.0552,  ...,  0.7551,  1.9583,  0.8927],
        ...,
        [ 2.8443,  0.3922, -2.1154,  ..., -2.2223, -0.9115,  1.2456],
        [ 1.0599,  0.8364, -0.8603,  ..., -0.1803,  0.7593, -2.2025],
        [-2.0571, -2.0313, -2.8774,  ...,  0.4580, -1.6490, -2.4690]], device='cuda:0')
None tensor([ 0.1524,  0.1286,  0.0933,  0.1070,  0.0905,  0.1577,  0.1286,
        -0.0236,  0.1499, -0.0638,  0.1461,  0.0990, -0.0370, -0.1710,
         0.1580,  0.0154,  0.0169, -0.1348, -0.0578,  0.1622, -0.1349,
         0.1647,  0.1606, -0.0570, -0.1010, -0.0545, -0.0209, -0.0544,
        -0.0385, -0.1209, -0.0495,  0.1178,  0.1159,  0.0017,  0.1328,
        -0.0586, -0.0387, -0.1375,  0.1198, -0.0730, -0.0602, -0.1442,
         0.1423, -0.1408,  0.0058, -0.1203, -0.0310,  0.0721,  0.1069,
        -0.14

### 3. Train the Agent with DDPG

In [ ]:
def ddpg(n_episodes=100, max_t=300, print_every=500):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        score = 0
        for t in range(1, max_t):
            #pdb.set_trace()
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations 
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished 
            agent.step(states, actions, rewards, next_states, dones)
            states = next_states
            score += np.mean(rewards)
            if max(dones):
                break 
                
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

C:\Users\mbgpcsk4\Dropbox (The University of Manchester)\D2019\University\Udacity\P2_Reacher_Submission\Udacity\MultiAgent\ddpg_agent.py:123: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)


Episode 14	Average Score: 0.07

### 4. Watch a Smart Agent!

In [4]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset()
for t in range(200):
    action = agent.act(state, add_noise=False)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 



### 6. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  